## TorchVision Object Detection

Adapted by Laurentius von Liechti from this tutorial: 
https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [2]:
import os

os.environ["KERAS_BACKEND"] = "torch"  # @param ["tensorflow", "jax", "torch"]

import os
import re
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import tensorflow_datasets as tfds
import tensorflow as tf
from torch.utils.data import Dataset
from torchvision import tv_tensors
from torchvision.io import read_image
from torchvision.transforms import v2 as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from utils.engine import train_one_epoch, evaluate
import utils.utils as utils


### Utils

Fetch these into the utils folder if they aren't already present.

* os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
* os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
* os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
* os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
* os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

In [3]:
train_image_dir = "../data/train/images/"
train_label_dir = "../data/train/labels/"
val_image_dir = "../data/valid/images/"
val_label_dir = "../data/valid/labels/"

label_array = ['Ambulance', 'Bus', 'Car', 'Motorcycle', 'Truck']

In [4]:
"""
Group3DataSet class for loading images and labels

__init__ - initializes the class with image and label directories
__len__ - returns the number of images
__getitem__ - returns the image and label at the given index
get_pic - returns the image at the given path
get_target - returns the target at the given path
load - loads the images and labels from the directories

Parameters:
    image_dir - the directory containing the images
    label_dir - the directory containing the labels
"""
class Group3Dataset(Dataset):
    IMG_SIZE = 416
    images = []
    targets = []

    """
    Initializes the class with image and label directories

    Parameters:
        image_dir - the directory containing the images
        label_dir - the directory containing the labels
    """
    def __init__(self, image_dir, label_dir, transforms=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.load(image_dir, label_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        target = self.targets[idx]
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target

    """
    Returns the image at the given path

    Parameters:
        img_path - the path to the image

    Returns:
        dimg - the image tensor
    """
    def get_pic(self, img_path):
        img = read_image(img_path)
        tvimg = tv_tensors.Image(img)
        dimg = torchvision.transforms.v2.functional.to_dtype(tvimg, torch.float32, scale=True)
        return dimg

    """
    Returns the target at the given label_path for the given img_path

    Converts the [x/y]center/width/height label to a tensor and 
    calculates the bounding box coordinates for each label in the label file

    It returns a dictionary containing the boxes, labels, image_id, area, 
    and iscrowd expected by the FasterRCNN model

    Parameters:
        img_path - the path to the image
        label_path - the path to the label

    Returns:
        dict - a target dictionary
    """
    def get_target(self, img_path, label_path):
        rows = open(label_path).read().strip().split("\n")
        boxes = []
        labels = []
        areas = []
        for row in rows:
            label, x_center, y_center, width, height = re.split(r'\s+', row)
            x_start = int((float(x_center) - float(width)/2)*self.IMG_SIZE)
            y_start = int((float(y_center) - float(height)/2)*self.IMG_SIZE)
            x_end = int((float(x_center) + float(width)/2)*self.IMG_SIZE)
            y_end = int((float(y_center) + float(height)/2)*self.IMG_SIZE)
            boxes.append([x_start, y_start, x_end, y_end])
            labels.append(int(label) + 1) # 0 is reserved for background
            areas.append((x_end - x_start) * (y_end - y_start))
        return {
            'boxes': tv_tensors.BoundingBoxes(
                boxes,
                format=tv_tensors.BoundingBoxFormat.XYXY, 
                canvas_size=(self.IMG_SIZE, self.IMG_SIZE)), 
            'labels': torch.tensor(labels, dtype=torch.int64), 
            'image_id': img_path,
            'area': torch.Tensor(areas),
            'iscrowd': torch.zeros((len(labels),), dtype=torch.int64)
            }

    def load(self, image_dir, label_dir):
        image_files = os.listdir(image_dir)
        label_files = os.listdir(label_dir)
        image_files.sort()
        label_files.sort()
        self.images = []
        self.targets = []
        
        for i, img_file in enumerate(image_files):
            image_path = image_dir + img_file
            label_path = label_dir + label_files[i]
            self.images.append(self.get_pic(image_path))
            self.targets.append(self.get_target(image_path, label_path))
            
        return self.images, self.targets

In [5]:
def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

### Test the custom dataset

* Use a small batch size and simple model to test our custom dataset to ensure it has all the required data structures in the right format for training and testing, end-to-end

In [6]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

train_ds = Group3Dataset(train_image_dir, train_label_dir, get_transform(train=True))

data_loader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    collate_fn=utils.collate_fn
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])

{'loss_classifier': tensor(0.3110, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0459, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0152, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0067, grad_fn=<DivBackward0>)}
{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>)}


## Train and Evaluate FasterRCNN

In [8]:
# Leaving out MPS because of the float32 limitation
# if torch.backends.mps.is_available():
#     device = torch.device("mps")
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Device: {device}")

# our dataset has two classes only - background and person
num_classes = 6
# use our dataset and defined transformations
dataset = Group3Dataset(train_image_dir, train_label_dir, get_transform(train=True))
dataset_test = Group3Dataset(val_image_dir, val_label_dir, get_transform(train=False))

# Skip this because we already have a train/test split for consistent model comparison
# split the dataset in train and test set
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    collate_fn=utils.collate_fn
)

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = len(label_array) + 1  # # classes + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# let's train it just for 2 epochs
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

torch.save(model.state_dict(), "model_factrcnn.pth")

Device: cpu
Epoch: [0]  [ 0/25]  eta: 0:01:01  lr: 0.000213  loss: 2.3401 (2.3401)  loss_classifier: 1.8810 (1.8810)  loss_box_reg: 0.4292 (0.4292)  loss_objectness: 0.0251 (0.0251)  loss_rpn_box_reg: 0.0048 (0.0048)  time: 2.4756  data: 0.0009
Epoch: [0]  [10/25]  eta: 0:00:35  lr: 0.002294  loss: 1.2958 (1.3179)  loss_classifier: 0.9373 (1.0145)  loss_box_reg: 0.1595 (0.2590)  loss_objectness: 0.0154 (0.0350)  loss_rpn_box_reg: 0.0058 (0.0094)  time: 2.3505  data: 0.0005
Epoch: [0]  [20/25]  eta: 0:00:11  lr: 0.004376  loss: 0.6196 (1.0702)  loss_classifier: 0.3612 (0.7524)  loss_box_reg: 0.2195 (0.2820)  loss_objectness: 0.0173 (0.0274)  loss_rpn_box_reg: 0.0065 (0.0084)  time: 2.3324  data: 0.0004
Epoch: [0]  [24/25]  eta: 0:00:02  lr: 0.005000  loss: 0.5603 (0.9939)  loss_classifier: 0.2846 (0.6709)  loss_box_reg: 0.2014 (0.2781)  loss_objectness: 0.0185 (0.0354)  loss_rpn_box_reg: 0.0059 (0.0095)  time: 2.3280  data: 0.0004
Epoch: [0] Total time: 0:00:58 (2.3378 s / it)
creating 